<a href="https://colab.research.google.com/github/GabrielAlencar5/JogoDeSorteio/blob/main/JogoDeSorteio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai
import google.generativeai as genai
from google.colab import userdata

API_KEY=userdata.get('API_KEY')
genai.configure(api_key=API_KEY)

gemini_model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
import random
import google.generativeai as genai
import os
from google.colab import userdata

API_KEY = os.getenv("API_KEY")

if not API_KEY:
    try:
        API_KEY = userdata.get('API_KEY')
        if not API_KEY:
            API_KEY = userdata.get('GEMINI_API_KEY')
    except:
        pass

if not API_KEY:
    print("ALERTA: A Chave API não foi encontrada.")
    print("Por favor, adicione-a no ícone de cadeado do Colab como 'GEMINI_API_KEY' ou 'API_KEY'.")

genai.configure(api_key=API_KEY)

try:
    GEMINI_FLASH = genai.GenerativeModel('gemini-2.5-flash')
except Exception as e:
    print(f"Erro ao inicializar o cliente Gemini: {e}")
    GEMINI_FLASH = None


def gerar_frase_motivacional():
    if not GEMINI_FLASH:
        return "Não desista! A sorte pode mudar na próxima rodada!"

    try:
        prompt = "Gere uma frase motivacional curta para incentivar alguém a continuar jogando após perder várias rodadas consecutivas. A frase deve ser de ânimo e não incentivar apostas irresponsáveis. Responda em português."
        response = GEMINI_FLASH.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return "Não desista! A sorte pode mudar na próxima rodada!"

DEPOSITO_MINIMO = 50.0
APOSTA_MINIMA = 5.0
APOSTA_MAXIMA = 30000.0
MULTIPLICADOR_PREMIO = 1.7
LUCRO_OBJETIVO = 100000.0
NUM_MIN = 0
NUM_MAX = 100

def sorteio():
    return random.randint(NUM_MIN, NUM_MAX)

def soma_um_com_wrap(n):
    novo = n + 1
    if novo > NUM_MAX:
        novo = NUM_MIN
    return novo

def jogar():
    print("=== JOGO DA SORTE: ENCONTRE O PRÊMIO MÁXIMO! ====")

    try:
        deposito = float(input(f"Para começar, faça um depósito mínimo de R${DEPOSITO_MINIMO:.2f}: ").strip())
    except ValueError:
        print("Valor de depósito inválido. O jogo será encerrada.")
        return

    if deposito < DEPOSITO_MINIMO:
        print("Depósito insuficiente. O jogo será encerrada.")
        return

    saldo_jogador = deposito
    lucro_maquina = 0.0
    jackpot_liberado = False
    perdas_consecutivas = 0
    parar_jogo = False

    while saldo_jogador >= APOSTA_MINIMA:
        if parar_jogo:
            break

        print(f"\nSeu saldo atual: R${saldo_jogador:.2f}")

        try:
            aposta = float(input(f"Faça sua aposta (mín R${APOSTA_MINIMA:.2f}, máx R${APOSTA_MAXIMA:.2f}): ").strip())
        except ValueError:
            print("Aposta inválida. Tente novamente.")
            continue

        if aposta < APOSTA_MINIMA or aposta > APOSTA_MAXIMA:
            print("Valor da aposta fora dos limites permitidos.")
            continue
        if aposta > saldo_jogador:
            print("Saldo insuficiente para esta aposta.")
            continue

        while True:
            try:
                numero_jogador = int(input(f"Escolha seu número da sorte entre {NUM_MIN} e {NUM_MAX}: ").strip())
            except ValueError:
                print("Número inválido. Tente novamente.")
                continue

            if numero_jogador < NUM_MIN or numero_jogador > NUM_MAX:
                print("Número fora do intervalo permitido. Tente novamente.")
                continue

            break

        if (not jackpot_liberado) and (lucro_maquina >= LUCRO_OBJETIVO):
            jackpot_liberado = True

        numero_sorteado_original = sorteio()

        if jackpot_liberado:
            numero_sorteado = numero_jogador
            print(f"O número sorteado foi: {numero_sorteado}")

        elif numero_jogador == numero_sorteado_original:
            numero_sorteado = soma_um_com_wrap(numero_sorteado_original)
            print(f"O número sorteado foi: {numero_sorteado}.")

        else:
            numero_sorteado = numero_sorteado_original
            print(f"O número sorteado foi: {numero_sorteado}")

        if numero_jogador == numero_sorteado and jackpot_liberado:
            premio = aposta * MULTIPLICADOR_PREMIO
            saldo_jogador += premio
            lucro_maquina -= (premio - aposta)

            print(f"\nParabéns! Você acertou e ganhou R${premio:.2f}.")

            jackpot_liberado = False
            perdas_consecutivas = 0

        else:
            saldo_jogador -= aposta
            lucro_maquina += aposta
            print("Você não acertou. Tente novamente!")
            perdas_consecutivas += 1

            if perdas_consecutivas % 3 == 0:
                print(f"\n{gerar_frase_motivacional()}\n")

        if saldo_jogador < APOSTA_MINIMA:
            print("\nSeu saldo é insuficiente para continuar jogando. Fim de jogo.")
            break

        while True:
            continuar = input("Deseja continuar jogando? (s/n): ").strip().lower()
            if continuar == 's' or continuar == 'sim':
                break
            elif continuar == 'n' or continuar == 'não' or continuar == 'nao':
                print("\nVocê escolheu parar de jogar.")
                parar_jogo = True
                break
            else:
                print("Opção inválida. Por favor, digite 's' ou 'n'.")

    print("\n=== FIM DO JOGO ====")
    print(f"Seu saldo final: R${saldo_jogador:.2f}")

if __name__ == "__main__":
    jogar()